Feature selection

BERT integration

In [1]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

In [5]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install AutoModel

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB 217.9 kB/s eta 0:00:08
   - -------------------------------------- 0.1/1.7 MB 491.5 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.7 MB 1.1 MB/s eta 0:00:02
   -------------------- ------------------- 0.9/1.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


distilbert-base-uncased because:
- pre-trained BERT model
- text classification
- distilled BERT model for smaller dataset
- slightly lesser performance than bert-base-uncased
- switch to bert-base-uncased for higher accuracy and if we have more computational resources; it will be slower

maybe not BERT? Any BERT models max text length is 512 and the news texts are longer


In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import torch
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from datasets import Dataset
from transformers import AutoTokenizer # Hugging Face Transformers
from transformers import AutoModel
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam


In [14]:
news_df = pd.read_csv('Data/raw_data/fake_or_real_news.csv') # TODO: Get cleaned data instead

label_encoder = preprocessing.LabelEncoder()
news_df['label'] = label_encoder.fit_transform(news_df['label'].tolist())

In [15]:
train_df, test_df = train_test_split(news_df, test_size=0.2)

In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModel, AdamW 
# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 
model = AutoModel.from_pretrained("distilbert-base-uncased") # Example news data (replace with your actual dataset) 
inputs = tokenizer(news_df['title'].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=512)
optimizer = AdamW(model.parameters(), lr=5e-5) # TODO: vary learning rates
model.train()  # Set the model to training mode 
for epoch in range(3): #TODO: vary epochs
    optimizer.zero_grad()  # Zero out previous gradients 
    # Forward pass: Get model outputs 
    with torch.no_grad(): # Since we're just extracting embeddings, no need to compute gradients 
        outputs = model(**inputs) # Extract the CLS token embeddings (usually the first token) 
        cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(cls_embeddings, news_df['label'].to_list(), test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000) # TODO: try knn
clf.fit(X_train, y_train)

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7584846093133386


In [33]:
df = pd.DataFrame([clf.predict(X_test)])
df

,0,1,2,3,4,5,6,7,8,9,...,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266
0,0,0,1,0,0,0,1,1,0,0,...,1,1,0,1,1,0,1,1,1,1
